In [1]:
# Initialise a spark session
import pandas as pd
from collections import Counter
import os
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator


spark = (
    SparkSession.builder.appName("RF Model")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "16g")  # Increase driver memory
    .config("spark.executor.memory", "16g")  # Increase executor memory
    .config("spark.executor.instances", "4")  # Increase the number of executor instances
    .config("spark.driver.maxResultSize", "2g")
    .config("spark.sql.shuffle.partitions", "200") \
    .getOrCreate()
)

24/09/25 14:32:58 WARN Utils: Your hostname, Melissas-MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 192.168.0.3 instead (on interface en0)
24/09/25 14:32:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/25 14:32:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Read transaction file
transactions = spark.read.parquet('../data/curated/flagged_fraud')
transactions = transactions.filter(F.col("is_fraud") != True) # Exclude transactions marked as fraud

In [3]:
# Aggregating monthly revenue for each merchant
monthly_revenue_df = transactions.groupBy('merchant_abn', 'order_month_year', 'name_merchant').agg(
    F.sum('dollar_value').alias('monthly_revenue'),
    F.count('order_id').alias('transaction_count'),
    F.avg('fraud_probability_merchant').alias('avg_fraud_probability_merchant'),
    F.first('tags').alias('merchant_tags')  # Assuming tags are constant per merchant
)
    
# Aggregating consumer-level features (most common state and gender for each merchant)

# Most common consumer state per merchant
consumer_state_mode = transactions.groupBy('merchant_abn', 'state_consumer').count() \
    .withColumn('row_num', F.row_number().over(Window.partitionBy('merchant_abn').orderBy(F.desc('count')))) \
    .filter(F.col('row_num') == 1) \
    .select('merchant_abn', 'state_consumer')

# Most common consumer gender per merchant
consumer_gender_mode = transactions.groupBy('merchant_abn', 'gender_consumer').count() \
    .withColumn('row_num', F.row_number().over(Window.partitionBy('merchant_abn').orderBy(F.desc('count')))) \
    .filter(F.col('row_num') == 1) \
    .select('merchant_abn', 'gender_consumer')

# Average Unemployment Rate per Merchant Month-Year
transactions = transactions.withColumn("unemployment_rate_numeric", F.col("unemployment_rate").cast("float"))

unemployment_agg = transactions.groupBy('merchant_abn', 'order_month_year').agg(
    F.avg('unemployment_rate_numeric').alias('avg_unemployment_rate')
)

24/09/25 14:33:09 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
# Joining Datasets
monthly_revenue_df = monthly_revenue_df.join(consumer_state_mode, on='merchant_abn', how='left') \
                                      .join(consumer_gender_mode, on='merchant_abn', how='left')

# Join with unemployment data on both 'merchant_abn' and 'order_month_year'
monthly_revenue_df = monthly_revenue_df.join(unemployment_agg, on=['merchant_abn', 'order_month_year'], how='left')

# Show the final dataframe
monthly_revenue_df.show(5)

+------------+----------------+--------------------+------------------+-----------------+------------------------------+--------------------+--------------+---------------+---------------------+
|merchant_abn|order_month_year|       name_merchant|   monthly_revenue|transaction_count|avg_fraud_probability_merchant|       merchant_tags|state_consumer|gender_consumer|avg_unemployment_rate|
+------------+----------------+--------------------+------------------+-----------------+------------------------------+--------------------+--------------+---------------+---------------------+
| 10023283211|          Aug-21|       Felis Limited|15807.479921460477|               86|              56.2429773174978|((furniture, home...|           NSW|           Male|    80.79883797224178|
| 10023283211|          Mar-21|       Felis Limited| 9076.307821688919|               40|             56.40749878739966|((furniture, home...|           NSW|           Male|     78.1724992275238|
| 10023283211|          M

In [5]:
# Creating lag features to include previous month's revenue
window_spec = Window.partitionBy('merchant_abn').orderBy('order_month_year')

# Lagging features: Previous month's revenue
monthly_revenue_df = monthly_revenue_df.withColumn(
    'previous_month_revenue', F.lag('monthly_revenue', 1).over(window_spec)
)

# Calculate revenue growth (percentage change)
monthly_revenue_df = monthly_revenue_df.withColumn(
    'revenue_growth',
    F.when(F.col('previous_month_revenue') > 0, 
           (F.col('monthly_revenue') - F.col('previous_month_revenue')) / F.col('previous_month_revenue'))
    .otherwise(F.lit(0))  # Fill with 0 if there is no previous revenue
)

# Fill NA values for first month with 0 (no previous data available)
monthly_revenue_df = monthly_revenue_df.fillna({'previous_month_revenue': 0, 'revenue_growth': 0})


monthly_revenue_df = monthly_revenue_df.fillna(0)  # Filling NA values for first month
monthly_revenue_df.show(5)

+------------+----------------+-------------+------------------+-----------------+------------------------------+--------------------+--------------+---------------+---------------------+----------------------+--------------------+
|merchant_abn|order_month_year|name_merchant|   monthly_revenue|transaction_count|avg_fraud_probability_merchant|       merchant_tags|state_consumer|gender_consumer|avg_unemployment_rate|previous_month_revenue|      revenue_growth|
+------------+----------------+-------------+------------------+-----------------+------------------------------+--------------------+--------------+---------------+---------------------+----------------------+--------------------+
| 10023283211|          Apr-21|Felis Limited|   9221.4058068711|               47|             56.03849374950703|((furniture, home...|           NSW|           Male|    74.54042625427246|                   0.0|                 0.0|
| 10023283211|          Aug-21|Felis Limited|15807.479921460477|        

In [6]:
from pyspark.ml.feature import StandardScaler, StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline

# StringIndexing categorical columns (merchant_tags, consumer_state, gender_consumer)
indexers = [
    StringIndexer(inputCol='merchant_tags', outputCol='merchant_tags_indexed', handleInvalid='keep'),
    StringIndexer(inputCol='state_consumer', outputCol='state_consumer_indexed', handleInvalid='keep'),
    StringIndexer(inputCol='gender_consumer', outputCol='gender_consumer_indexed', handleInvalid='keep')
]

# OneHotEncoding indexed columns
encoders = [
    OneHotEncoder(inputCol='merchant_tags_indexed', outputCol='merchant_tags_encoded'),
    OneHotEncoder(inputCol='state_consumer_indexed', outputCol='state_consumer_encoded'),
    OneHotEncoder(inputCol='gender_consumer_indexed', outputCol='gender_consumer_encoded')
]

# VectorAssembler to combine numeric features into a single feature vector
assembler = VectorAssembler(
    inputCols=[
        'monthly_revenue', 'transaction_count', 'avg_fraud_probability_merchant', 'avg_unemployment_rate',
        'merchant_tags_encoded', 'state_consumer_encoded', 'gender_consumer_encoded', 'revenue_growth'
    ], 
    outputCol='features'
)

# Standardizing the numeric features
scaler = StandardScaler(inputCol='features', outputCol='scaled_features')

pipeline = Pipeline(stages=indexers + encoders + [assembler, scaler])

# Fit the pipeline to the dataset
model_pipeline = pipeline.fit(monthly_revenue_df)

final_df = model_pipeline.transform(monthly_revenue_df)

final_df.select('merchant_abn', 'order_month_year', 'scaled_features').show(5)

24/09/25 14:34:04 WARN DAGScheduler: Broadcasting large task binary with size 57.4 MiB
24/09/25 14:34:12 WARN DAGScheduler: Broadcasting large task binary with size 57.4 MiB
24/09/25 14:34:25 WARN DAGScheduler: Broadcasting large task binary with size 23.3 MiB


+------------+----------------+--------------------+
|merchant_abn|order_month_year|     scaled_features|
+------------+----------------+--------------------+
| 10023283211|          Apr-21|(439966,[0,1,2,3,...|
| 10023283211|          Aug-21|(439966,[0,1,2,3,...|
| 10023283211|          Dec-21|(439966,[0,1,2,3,...|
| 10023283211|          Feb-22|(439966,[0,1,2,3,...|
| 10023283211|          Jan-22|(439966,[0,1,2,3,...|
+------------+----------------+--------------------+
only showing top 5 rows



In [7]:
train_data, test_data = final_df.randomSplit([0.8, 0.2], seed=42)

In [8]:
# Define the RandomForestRegressor model
rf = RandomForestRegressor(featuresCol='scaled_features', labelCol='monthly_revenue')

# Create a parameter grid for hyperparameter tuning
paramGrid = (ParamGridBuilder()
             .addGrid(rf.numTrees, [20, 50, 100])     # Tune number of trees
             .addGrid(rf.maxDepth, [5, 10, 15])       # Tune max depth of the tree
             .addGrid(rf.maxBins, [32, 50])           # Tune number of bins
             .build())

# Define an evaluator based on RMSE
evaluator = RegressionEvaluator(labelCol='monthly_revenue', predictionCol='prediction', metricName='rmse')

# Set up cross-validation with 5 folds
crossval = CrossValidator(estimator=rf,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=5)  # 5-fold cross-validation

In [9]:
# Fit the cross-validated model
cvModel = crossval.fit(train_data)

# Make predictions on the test data
predictions = cvModel.transform(test_data)

24/09/25 14:35:05 WARN DAGScheduler: Broadcasting large task binary with size 80.2 MiB
24/09/25 14:36:42 WARN DAGScheduler: Broadcasting large task binary with size 80.2 MiB
24/09/25 14:36:44 WARN DAGScheduler: Broadcasting large task binary with size 80.2 MiB
24/09/25 14:38:07 WARN DAGScheduler: Broadcasting large task binary with size 84.5 MiB
24/09/25 14:40:53 WARN DAGScheduler: Broadcasting large task binary with size 1728.3 KiB
24/09/25 14:40:56 WARN DAGScheduler: Broadcasting large task binary with size 97.0 MiB
24/09/25 14:41:10 WARN MemoryStore: Not enough space to cache rdd_407_2 in memory! (computed 3.6 GiB so far)
24/09/25 14:41:11 WARN MemoryStore: Not enough space to cache rdd_407_0 in memory! (computed 3.6 GiB so far)
24/09/25 14:41:11 WARN MemoryStore: Not enough space to cache rdd_407_8 in memory! (computed 109.1 MiB so far)
24/09/25 14:41:11 WARN BlockManager: Persisting block rdd_407_0 to disk instead.
24/09/25 14:41:11 WARN BlockManager: Persisting block rdd_407_2 to

KeyboardInterrupt: 

In [ ]:
# Evaluate RMSE on the test data
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Evaluate R-squared on the test data
r2_evaluator = RegressionEvaluator(labelCol='monthly_revenue', predictionCol='prediction', metricName='r2')
r2 = r2_evaluator.evaluate(predictions)
print(f"R-squared: {r2}")

# Show a few predictions
predictions.select('merchant_abn', 'order_month_year', 'monthly_revenue', 'revenue_growth', 'prediction').show(5)

# Best model's parameters
best_model = cvModel.bestModel
print(f"Best numTrees: {best_model.getNumTrees}")
print(f"Best maxDepth: {best_model.getMaxDepth}")
print(f"Best maxBins: {best_model.getMaxBins}")

In [ ]:
from pyspark.sql.types import DateType
from dateutil.relativedelta import relativedelta
from datetime import datetime

# Step 1: Parse the order_month_year column to a proper date format
monthly_revenue_df = monthly_revenue_df.withColumn(
    'order_month_year_date', F.to_date(F.concat(F.lit('01-'), F.col('order_month_year')), 'dd-MMM-yy')
)

# Get the most recent month per merchant
window_spec = Window.partitionBy('merchant_abn').orderBy(F.desc('order_month_year_date'))
latest_merchant_data = monthly_revenue_df.withColumn('row_num', F.row_number().over(window_spec)) \
                                         .filter(F.col('row_num') == 1) \
                                         .drop('row_num')

In [ ]:
next_month = 'Aug-24'
future_month_df = spark.createDataFrame([(next_month,)], ['future_order_month_year'])
future_data = latest_merchant_data.crossJoin(future_month_df)

In [ ]:
future_data = model_pipeline.transform(future_data)
future_data = cvModel.transform(future_data)
future_predictions = future_data.select('merchant_abn', 'merchant_name','merchant_tags','future_order_month_year', 'prediction')
future_predictions = future_predictions.withColumnRenamed('prediction', 'projected_revenue')
future_predictions.show(5)

In [ ]:
top_10_predictions = future_predictions.orderBy(F.col('projected_revenue').desc())

# Show the top 10 merchants by predicted revenue
top_10_predictions.select('merchant_abn', 'merchant_name', 'projected_revenue').show(10)

In [10]:
spark.stop()

24/09/25 14:44:01 ERROR Instrumentation: org.apache.spark.SparkException: Job 92 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$cleanUpAfterSchedulerStop$1(DAGScheduler.scala:1253)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$cleanUpAfterSchedulerStop$1$adapted(DAGScheduler.scala:1251)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:79)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:1251)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:3087)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:84)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$stop$3(DAGScheduler.scala:2973)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1375)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:2973)
	at org.apache.spark.SparkContext.$anonfun$stop$12(SparkContext.scala:2263)
	at org.apache.spark.util.Utils$.tryL